This notebook provide a technical background of the multi-family apartment building model.

# Building Model

The thermal model is a based on a four-zone ETP model, where the following zones variables are defined:

Table 1: Zone variables

| Zone Name            | Temperature | Mass   | Heat   |
| :------------------- | :---------: | :----: | :----: |
| Occupied apartment   | $T_A$       | $C_A$  | $Q_A$  |
| Unoccupied apartment | $T_U$       | $C_U$  | $Q_U$  |
| Building core        | $T_C$       | $C_C$  | $Q_C$  |
| Building mass        | $T_M$       | $C_M$  |   -    |
| Outdoor air          | $T_O$       |   -    |   -    |

The model assumes that every zone is connected to every other zone with the following zone-zone conductance parameters defined

Table 2: Zone-zone conductance

| Conductance | Connected zones                       |
| ----------- | ------------------------------------- |
| $U_{OA}$    | Occupied apartment & outdoor air      |
| $U_{OU}$    | Unoccupied apartment & outdoor air    |
| $U_{OC}$    | Building core & outdoor air           |
| $U_{OM}$    | Building mass & outdoor air           |
| $U_{AU}$    | Occupied & unoccupied apartments      |
| $U_{AC}$    | Occupied apartments & building core   |
| $U_{AM}$    | Occupied apartments & building mass   |
| $U_{UC}$    | Uncccupied apartments & building core |
| $U_{UM}$    | Unoccupied apartments & building mass |
| $U_{CM}$    | Building core & building mass         |

The thermal dynamics are defined by Equation (1)

$$
    \begin{bmatrix} 
        \dot T_A 
    \\ 
        \dot T_U 
    \\
        \dot T_C
    \\
        \dot T_M
    \end{bmatrix}
    =
    \begin{bmatrix}
        -\frac{U_{AU}+U_{AC}+U_{AM}}{C_A} & \frac{U_{AU}}{C_A} & \frac{U_{AC}}{C_A} & \frac{U_{AM}}{C_A}
    \\
        \frac{U_{AU}}{C_U} & -\frac{U_{AU}+U_{UC}+U_{UM}}{C_U} & \frac{U_{UC}}{C_U} & \frac{U_{UM}}{C_U}
    \\
        \frac{U_{AC}}{C_C} & \frac{U_{UC}}{C_C} & -\frac{U_{AC}+U_{UC}+U_{CM}}{C_C} & \frac{U_{CM}}{C_C}
    \\
        \frac{U_{AM}}{C_M} & \frac{U_{UM}}{C_M} & \frac{U_{CM}}{C_M} & -\frac{U_{AM}+U_{UM}+U_{CM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        T_A
    \\
        T_U
    \\
        T_C
    \\
        T_M
    \end{bmatrix}
    +
    \begin{bmatrix}
        \frac{1}{C_A} & 0 & 0 & \frac{U_{OA}}{C_A}
    \\
        0 & \frac{1}{C_U} & 0 & \frac{U_{OU}}{C_U}
    \\
        0 & 0 & \frac{1}{C_C} & \frac{U_{OC}}{C_C}
    \\
        0 & 0 & 0 & \frac{U_{OM}}{C_M}
    \end{bmatrix}
    \begin{bmatrix}
        Q_A
    \\
        Q_U
    \\
        Q_C
    \\
        T_O
    \end{bmatrix}
    \qquad (1)
$$

Equation (1) is represented more simply as

$$
    \boxed { \dot {\textbf T} = \textbf A \textbf T + \textbf B_1 \textbf q + \textbf B_2 \textbf u } \qquad (2)
$$

where $\textbf q$ is the ambient heat gain/loss, and $\textbf u$ is the controlled heat gain/loss, with

$$
    \textbf B_1 = \begin{bmatrix}
        U_{OA} \over C_A & 1 \over C_A & 1 \over C_A & 1 \over C_A & 0 & 0 & 0
    \\
        U_{OU} \over C_U & 0 & 0 & 0 & 1 \over C_U & 0 & 0
    \\
        U_{OC} \over C_C & 0 & 0 & 0 & 0 & 1 \over C_C & 1 \over C_C
    \\
        U_{OM} \over C_M & 0 & 0 & 0 & 0 & 0 & 0
    \end{bmatrix}
    \qquad \textrm{and} \qquad
    \textbf B_2 = \begin{bmatrix}
        1 \over C_A & 0 & 0
    \\
        0 & 1 \over C_U & 0 
    \\
        0 & 0 & 1 \over C_C
    \\
        0 & 0 & 0
    \end{bmatrix}
$$

and

$$
    \textbf q = \begin{bmatrix} T_O \\ Q_{AS} \\ Q_{AV} \\ Q_{AE} \\ Q_{US} \\ Q_{CS} \\ Q_{CV} \end{bmatrix}
    \qquad \textrm{and} \qquad
    \textbf u = \begin{bmatrix} Q_{AH} \\ Q_{UH} \\ Q_{CH} \end{bmatrix}.
$$

The heat gains for each zones are defined as follows

Table 3: Occupied apartment heat gains

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{AH}$  |
| Solar gain       | $Q_{AS}$  |
| Ventilation gain | $Q_{AV}$  |
| End-use gains    | $Q_{AE}$  |

Table 4: Unoccupied apartment heat gains 

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{UH}$  |
| Solar gain       | $Q_{US}$  |

Table 5: Building core heat gains

| Heat source      | Parameter |
| ---------------- | --------- |
| HVAC             | $Q_{CH}$  |
| Solar gain       | $Q_{CS}$  |
| Ventilation gain | $Q_{CV}$  |

## End-uses

The following end-uses contribute to heat gain in building zones and are driven by schedules

Table 6: Zone end-use schedule source

| End-use     | $Q_{AE}$      | $Q_{UE}$     | $Q_{CE}$     |
| ----------- | ------------- | ------------ | ------------ |
| Lights      | `lighting`    |      -       | `hallway`    |
| Plugs       | `plugs`       |      -       |      -       |
| Cooking     | `cooking`     |      -       |      -       |
| Dishwashing | `dishwashing` |      -       |      -       |
| Washing     | `washing`     |      -       |      -       | 
| Drying      | `drying`      |      -       |      -       |
| Hotwater    | `hotwater`    |      -       |      -       |


## HVAC Controls

The following thermostat setpoints are use to control HVAC

| Zone       | $T_{AH}$ | $T_{AC}$ |
| ---------- | -------- | -------- |
| Occupied   | $T_{AH}$ | $T_{AC}$ |
| Unccupied  | $T_{UH}$ | $T_{UC}$ |
| Core       | $T_{CH}$ | $T_{CC}$ |

# Solution Method

The model assumes that the setpoints are maintained in the controlled zone, and computes the heating/cooling load required by setting Equation (2) equal to zero.  However, if the heating/cooling load exceeds the capacity of the central plant or the HVAC units, then the power will be saturated, and the Equation (2) will provide the temperature changes, if any, for each zone.

## Step 1 - Compute the equilibrium load

Solve Equation (2) for the augmented $\textbf u$ given $\dot{\textbf T}=0$, i.e.,

$$
    \textbf u = -\textbf B_2^+ ( \textbf A \textbf T + \textbf B_1 \textbf q )
$$

## Step 2 - Constrain the heat gain/loss

The value of $\textbf u$ may exceed the limits of the HVAC system, in which can they must be constrained such that element-wise

$$
    \textbf u^* = \textbf u \qquad \mathrm{s.t} \qquad \underline {\textbf u} \le {\textbf u} \le \overline {\textbf u}
$$

where $\underline {\textbf u}$ is the maximum cooling capacity and $\overline {\textbf u}$ is the maximum heating capacity of the HVAC system.  

## Step 3 - Compute the temperature change

Given the constrained values $\textbf u^*$, we update the zone temperatures using Equation (2):

$$
    \dot {\textbf T} = \textbf{A} \textbf{T} + \textbf B_1 \textbf q + \textbf B_2 \textbf u^*
$$

where

$$
    \textbf T = \begin{bmatrix} T_A \\ T_U \\ T_C \\ T_M \end{bmatrix}
$$